In [1]:
import ccxt
import json
from pprint import pprint
import requests
from ddlgenerator.ddlgenerator import Table
from sqlalchemy.sql import text
from sqlalchemy import create_engine
import pandas as pd
import pandas.io.sql as psql

In [2]:
url = 'https://bittrex.com/api/v1.1/public/getmarketsummaries'
r = requests.get(url)
#print(json.loads(r.content))
data = json.loads(r.content.decode())

In [3]:
table = Table(data['result'])
sql = table.sql('postgresql', inserts=True)

In [4]:
sql

"DROP TABLE IF EXISTS generated_table0;\n\nCREATE TABLE generated_table0 (\n\tmarketname VARCHAR(9) NOT NULL, \n\thigh DECIMAL(12, 8) NOT NULL, \n\tlow DECIMAL(12, 8) NOT NULL, \n\tvolume DECIMAL(17, 8) NOT NULL, \n\t_last DECIMAL(12, 8) NOT NULL, \n\tbasevolume DECIMAL(15, 8) NOT NULL, \n\t_timestamp TIMESTAMP WITHOUT TIME ZONE NOT NULL, \n\tbid DECIMAL(12, 8) NOT NULL, \n\task DECIMAL(12, 8) NOT NULL, \n\topenbuyorders INTEGER NOT NULL, \n\topensellorders INTEGER NOT NULL, \n\tprevday DECIMAL(12, 8) NOT NULL, \n\tcreated TIMESTAMP WITHOUT TIME ZONE NOT NULL\n);\n\nINSERT INTO generated_table0 (marketname, high, low, volume, _last, basevolume, _timestamp, bid, ask, openbuyorders, opensellorders, prevday, created) VALUES ('BTC-2GIVE', 0.00000109, 0.00000107, 473669.62967021, 0.00000107, 0.51272628, '2018-05-28 00:39:19.490000', 0.00000107, 0.00000108, 152, 835, 0.00000107, '2016-05-16 06:44:15.287000');\nINSERT INTO generated_table0 (marketname, high, low, volume, _last, basevolume, _t

In [5]:
sql = sql.split('\n\n')

In [6]:
sql[2] = sql[2].split('\n')

In [7]:
mydb_lst = sql[0].split(' ')
mydb = mydb_lst[4]
mydb = mydb[:-1]

In [8]:
# create the session
engine = create_engine('sqlite:///:memory:')

In [9]:
res = engine.execute(sql[0])

In [10]:
res = engine.execute(sql[1])


In [11]:
sql[1]

'CREATE TABLE generated_table0 (\n\tmarketname VARCHAR(9) NOT NULL, \n\thigh DECIMAL(12, 8) NOT NULL, \n\tlow DECIMAL(12, 8) NOT NULL, \n\tvolume DECIMAL(17, 8) NOT NULL, \n\t_last DECIMAL(12, 8) NOT NULL, \n\tbasevolume DECIMAL(15, 8) NOT NULL, \n\t_timestamp TIMESTAMP WITHOUT TIME ZONE NOT NULL, \n\tbid DECIMAL(12, 8) NOT NULL, \n\task DECIMAL(12, 8) NOT NULL, \n\topenbuyorders INTEGER NOT NULL, \n\topensellorders INTEGER NOT NULL, \n\tprevday DECIMAL(12, 8) NOT NULL, \n\tcreated TIMESTAMP WITHOUT TIME ZONE NOT NULL\n);'

In [12]:
for x in sql[2]:
    res = engine.execute(x)

In [13]:
t = text("SELECT * FROM " + mydb + ";")
t = text("SELECT * FROM " + mydb + " WHERE marketname = 'BTC-TRX';")
res = engine.execute(t)

In [14]:
for _row in res: print (_row)

('BTC-TRX', 9.98e-06, 9.35e-06, 65419960.71024878, 9.5e-06, 630.92569777, '2018-05-28 00:41:13.297000', 9.48e-06, 9.5e-06, 2253, 7532, 9.76e-06, '2018-03-01 23:17:06.487000')


In [15]:
df = pd.DataFrame(engine.execute(t).fetchall())
df.columns = res.keys()

In [16]:
df

,marketname,high,low,volume,_last,basevolume,_timestamp,bid,ask,openbuyorders,opensellorders,prevday,created
0,BTC-TRX,0.00001,0.000009,6.541996e+07,0.00001,630.925698,2018-05-28 00:41:13.297000,0.000009,0.00001,2253,7532,0.00001,2018-03-01 23:17:06.487000
